## Query Synapses
This notebook demonstrates how to query synapses while proofreading is ongoing in H01 CAVE. 

In [53]:
from caveclient import CAVEclient
import datetime

In [54]:
url = "https://global.brain-wire-test.org/"
datastack = "h01_c3_flat"

In [55]:
client = CAVEclient(datastack_name=datastack, server_address=url)

In [61]:
client.materialize.get_versions()
print("Mat version: ", client.materialize.version)
client.materialize.get_tables()

Mat version:  429


['cells',
 'nucleus',
 'synapses',
 'proofreading_status_test',
 'synapses_using_sample_voxel_coords']

### Query Materialized Synapse Table
Querying the materialized synapse table leads to reproducable results since the table is a frozen snapshot in time. 

In [57]:
syn = client.materialize.synapse_query(
    post_ids=[864691132192363248],
    # pre_ids=[ADD YOUR ROOT ID],
)
syn.head()
print(len(syn))

62


### Live Synapse Queries
As proofreading is ongoing, neuron IDs change. If you want to query the most recent or any historic data, you can use the live synapse queries.

In [58]:
# check if root ID is the most recent root ID
root_id = 864691132192363248
now = datetime.datetime.now(datetime.timezone.utc)
is_latest = client.chunkedgraph.is_latest_roots([root_id], timestamp=now)
latest_id = client.chunkedgraph.get_latest_roots(root_id, timestamp=now)
print(is_latest) 
print(latest_id)

[False]
[864691132209940250]


In [60]:
synapse_table = client.info.get_datastack_info()['synapse_table']
df=client.materialize.query_table(synapse_table,
                                  timestamp = datetime.datetime.now(datetime.timezone.utc),
                                  filter_equal_dict = {'post_pt_root_id': latest_id[0]})
print(len(df))
df.head()

1328


,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,16727926,2024-04-11 23:26:47.169193+00:00,NaN,t,NaN,117125064771502458,864691132224138235,123950283268031227,864691132209940250,"[328058, 234952, 1800]","[377580, 230461, 2044]","[375758, 230749, 3693]"
1,16652642,2024-04-11 23:26:45.003008+00:00,NaN,t,NaN,126203458209448033,864691132277433667,123950283268031227,864691132209940250,"[394232, 240662, 3382]","[377580, 230461, 2044]","[395059, 238114, 4154]"
2,16776808,2024-04-11 23:26:49.048049+00:00,NaN,t,NaN,126343989874917806,864691132241474911,123950283268031227,864691132209940250,"[395066, 239458, 4019]","[377580, 230461, 2044]","[395004, 238224, 4171]"
3,16659732,2024-04-11 23:26:45.261722+00:00,NaN,t,NaN,126343852570181971,864691131940376673,123950283268031227,864691132209940250,"[395226, 238104, 4268]","[377580, 230461, 2044]","[395255, 238139, 4268]"
4,16666429,2024-04-11 23:26:45.261722+00:00,NaN,t,NaN,126343852570181841,864691132016386177,123950283268031227,864691132209940250,"[395199, 238122, 4257]","[377580, 230461, 2044]","[395195, 238216, 4246]"


### Disclaimer
For the H01 dataset, `pre_pt_position` and `post_pt_position` in the `synapses_using_sample_voxel_coords` table are random locations sampled from the respective supervoxel. Thus, these positions are not entirely representative of the pre and post synaptic sites and we discurage using those positions if accurate positions are needed. If you have more questions or need clarifications, please open an issue in this repository.